In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('data/userTrainData.csv', usecols=['user_id', 'business_id', 'stars', 'text'])

# General

In [7]:
print('Total users: ' + str(data['user_id'].nunique()))
print('Total businesses: ' + str(data['business_id'].nunique()))

Total users: 239007
Total businesses: 159168


# Missing Data

According to describe() function there are no missing data

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2303580 entries, 0 to 2303579
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      object 
 1   business_id  object 
 2   stars        float64
 3   text         object 
dtypes: float64(1), object(3)
memory usage: 70.3+ MB


# Number reviews per business

In [12]:
num_reviews_per_business = data.groupby('business_id')['user_id'].count().reset_index().rename(
    columns={'user_id': '#reviews'})
num_reviews_per_business.describe()

,#reviews
count,159168.000000
mean,14.472633
std,42.786480
min,1.000000
25%,2.000000
50%,4.000000
75%,11.000000
max,2595.000000


# Number reviews per users

Remove users that gave more than 100/500 ratings? 1487/28

In [13]:
num_reviews_per_users = data.groupby('user_id')['business_id'].count().reset_index().rename(
    columns={'business_id': '#reviews'})
num_reviews_per_users.describe()

,#reviews
count,239007.000000
mean,9.638128
std,19.139944
min,3.000000
25%,4.000000
50%,5.000000
75%,9.000000
max,2288.000000


# Review text length

In [15]:
data['text_len'] = data['text'].apply(lambda x: len(x))
data['text_len'].describe().reset_index()

,index,text_len
0,count,2.303580e+06
1,mean,6.549019e+02
2,std,5.916611e+02
3,min,1.000000e+00
4,25%,2.650000e+02
5,50%,4.750000e+02
6,75%,8.430000e+02
7,max,5.228000e+03
